In [ ]:
import ray
import time
import os
import cmlapi
import json

# Set the setup variables needed by CML APIv2
HOST = os.getenv("CDSW_API_URL").split(":")[0] + "://" + os.getenv("CDSW_DOMAIN")
USERNAME = os.getenv("CDSW_PROJECT_URL").split("/")[6]  # args.username  # "vdibia"
API_KEY = os.getenv("CDSW_APIV2_KEY")
PROJECT_NAME = os.getenv("CDSW_PROJECT")
PROJECT_ID=os.getenv("CDSW_PROJECT_ID")

cml = cmlapi.default_client(url=HOST,cml_api_key=API_KEY)

def set_environ(Cml,Item,Value):
  Project=Cml.get_project(os.getenv("CDSW_PROJECT_ID"))
  if Project.environment=='':
    Project_Environment={}
  else:
    Project_Environment=json.loads(Project.environment)
  Project_Environment[Item]=Value
  Project.environment=json.dumps(Project_Environment)
  Cml.update_project(Project,project_id=os.getenv("CDSW_PROJECT_ID"))

def get_environ(Cml,Item):
  Project=Cml.get_project(os.getenv("CDSW_PROJECT_ID"))
  Project_Environment=json.loads(Project.environment)
  return Project_Environment[Item]

In [ ]:
import modin.pandas as pd
from modin.config import BenchmarkMode
BenchmarkMode.put(True)
import urllib.request

In [ ]:
runtime_env = {
    "env_vars": {"__MODIN_AUTOIMPORT_PANDAS__": "1"}
}


ray.init(address=get_environ(cml,"RAY_ADDRESS"),runtime_env=runtime_env)

In [ ]:

url_path = "https://modin-test.s3.us-west-1.amazonaws.com/yellow_tripdata_2015-01.csv"



In [ ]:
urllib.request.urlretrieve(url_path, "taxi.csv")

In [ ]:
%time df = pd.read_csv("taxi.csv")

In [ ]:
%time isnull = df.isnull()

In [ ]:
%time rounded_trip_distance = df[["trip_distance"]].applymap(round)

In [ ]:
%time len(df)

In [ ]:
%time df.groupby('RateCodeID').mean('passenger_count').reset_index()

In [ ]:
df.dtypes